# Import package

In [1]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import loggers
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
%load_ext autoreload
%autoreload 2

# Setup environment

In [3]:
device = "cuda" if torch.cuda.is_available else "cpu"

def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return "%.2f" % size, power_labels[n] + 'bytes'

torch.cuda.get_device_name(), format_bytes(torch.cuda.get_device_properties(device).total_memory)

('GeForce GTX 1080 Ti', ('10.92', 'gigabytes'))

# Load model

In [4]:
from preprocessing.dataset import load_supervised_dataset

In [5]:
from trainer.supervised.baseline import BaselineTrainer
model = BaselineTrainer()
max_epoches = model.get_max_epoches()

>> Load from data frame


Using cache found in /home/kientiet/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


In [6]:
checkpoint_path = os.path.join(os.getcwd(), "checkpoint", model.model_name)
checkpoint_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    save_top_k = 5,
    verbose = True,
    monitor = 'kappa_score',
    mode = 'max'
)

checkpoint_path

/home/kientiet/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Checkpoint directory /home/kientiet/Documents/My Project/python/panda-kaggle/checkpoint/baseline/resnet50_32x4d exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


'/home/kientiet/Documents/My Project/python/panda-kaggle/checkpoint/baseline/resnet50_32x4d'

In [7]:
tb_logger = loggers.TensorBoardLogger('logs/', name = model.model_name)
trainer = pl.Trainer(checkpoint_callback = checkpoint_callback,
                    nb_sanity_val_steps = 0, 
                    max_epochs = max_epoches, 
                    gpus = -1, 
                    logger = tb_logger)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


# Learning finder

In [8]:
lr_finder = trainer.lr_find(model, min_lr = 1e-8, max_lr = 5., num_training = 200)

INFO:lightning:
    | Name                             | Type              | Params
-------------------------------------------------------------------
0   | loss_func                        | BCELoss           | 0     
1   | encoder                          | ResNetModel       | 22 M  
2   | encoder.encoder                  | Sequential        | 22 M  
3   | encoder.encoder.0                | Conv2d            | 9 K   
4   | encoder.encoder.1                | BatchNorm2d       | 128   
5   | encoder.encoder.2                | ReLU              | 0     
6   | encoder.encoder.3                | MaxPool2d         | 0     
7   | encoder.encoder.4                | Sequential        | 205 K 
8   | encoder.encoder.4.0              | Bottleneck        | 63 K  
9   | encoder.encoder.4.0.conv1        | Conv2d            | 8 K   
10  | encoder.encoder.4.0.bn1          | BatchNorm2d       | 256   
11  | encoder.encoder.4.0.conv2        | Conv2d            | 4 K   
12  | encoder.encoder.4.0.bn2   

> /home/kientiet/Documents/My Project/python/panda-kaggle/trainer/supervised/baseline.py(28)forward()
-> batch_size, image_batch, channels, height, width = images.shape


(Pdb)  n


> /home/kientiet/Documents/My Project/python/panda-kaggle/trainer/supervised/baseline.py(29)forward()
-> images = images.reshape(-1, channels, height, width)


(Pdb)  n


...

> /home/kientiet/Documents/My Project/python/panda-kaggle/trainer/supervised/baseline.py(38)forward()
-> logits = torch.sigmoid(logits)


(Pdb)  logits.shape


torch.Size([16, 5])


(Pdb)  exit


BdbQuit: 

In [ ]:
fig = lr_finder.plot(suggest = True)

In [8]:
model.max_lr = 2e-3
model.current_epoch = 0
model.configure_optimizers()

SGD (
Parameter Group 0
    base_momentum: 0.85
    dampening: 0
    initial_lr: 8e-05
    lr: 0.001999998892022334
    max_lr: 0.002
    max_momentum: 0.9
    min_lr: 8.000000000000001e-07
    momentum: 0.8500000277105259
    nesterov: False
    weight_decay: 5e-05
)

## Start to train

In [ ]:
trainer.fit(model)

INFO:lightning:
    | Name                             | Type              | Params
-------------------------------------------------------------------
0   | loss_func                        | CrossEntropyLoss  | 0     
1   | encoder                          | ResNetModel       | 22 M  
2   | encoder.encoder                  | Sequential        | 22 M  
3   | encoder.encoder.0                | Conv2d            | 9 K   
4   | encoder.encoder.1                | BatchNorm2d       | 128   
5   | encoder.encoder.2                | ReLU              | 0     
6   | encoder.encoder.3                | MaxPool2d         | 0     
7   | encoder.encoder.4                | Sequential        | 205 K 
8   | encoder.encoder.4.0              | Bottleneck        | 63 K  
9   | encoder.encoder.4.0.conv1        | Conv2d            | 8 K   
10  | encoder.encoder.4.0.bn1          | BatchNorm2d       | 256   
11  | encoder.encoder.4.0.conv2        | Conv2d            | 4 K   
12  | encoder.encoder.4.0.bn2   

>> Return trainloader
>> Return valloader
